In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold


### Loading data

There is an error with discussion_questions.csv and discussion_answers.csv which causes almost half the lines to be removed. This error is caused because of incorrect formating in the csv since commas in the discussion contents are being recognized as delimiters.

delete all `{\"dtdId\":\"discussion/1\",\"value\":\"<co-content>`

delete all `</co-content>\"}`

replace all `\\\" with ""`

In [118]:
disc_qs = pd.read_csv("MTC508/discussion_questions.csv")
disc_qs_votes = pd.read_csv("MTC508/discussion_question_votes.csv")
disc_qs_followings = pd.read_csv("MTC508/discussion_question_followings.csv")
disc_ans = pd.read_csv("MTC508/discussion_answers.csv")
disc_ans_votes = pd.read_csv("MTC508/discussion_answer_votes.csv")
users = pd.read_csv("MTC508/users.csv")
target = pd.read_csv("MTC508/MTC508_roster_outputlabel_Jaeyoon.csv")
ethnicity = pd.read_csv("MTC508/MTC508_roster_output_label_Jaeyoon.csv")
survey = pd.read_csv("MTC508/MTC508_roster_output_label_race_gender_from_survey_Jaeyoon.csv")

Select important column from the user table

In [119]:
# selected_user_columns = ["umich_user_id", "country_cd", "reported_or_inferred_gender"]
selected_user_columns = ["umich_user_id", "country_cd", "reported_or_inferred_gender", "educational_attainment"]
# selected_user_columns = ["umich_user_id"]

df = pd.DataFrame(users)[selected_user_columns]

In [120]:
gender = survey[['umich_user_id', 'gender_508']]

df = pd.merge(df, gender, on='umich_user_id', how='left') 
df.head()

,umich_user_id,country_cd,reported_or_inferred_gender,educational_attainment,gender_508
0,d8494a39cae7ac5417d491df43f8296a285b3cd6,US,female,BACHELOR_DEGREE,NaN
1,9c3e969ace3d1278a5e3ad42fa80dfc20379aaf5,US,female,PROFESSIONAL_DEGREE,NaN
2,c91dab5c98bead9e683d15e9e2cc2c4584a8e626,US,female,NaN,NaN
3,2246b3c08e2f3ee5d17f125ccc70baf3691667f9,US,female,BACHELOR_DEGREE,Female
4,1f8331ad9f0125b0fc85a1c4ff28f9c198b6fca2,US,NaN,NaN,NaN


Get the total number of discussion answers a user has posted

In [121]:
answer_counts = disc_ans.groupby('umich_discussions_user_id').size().reset_index(name='answer_count')
df = pd.merge(df, answer_counts, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')
df.drop('umich_discussions_user_id', axis=1, inplace=True)
df['answer_count'].fillna(0, inplace=True)

Get the average length of a discussion answer by a user

In [122]:
# Function to calculate the average answer length for a user
def calculate_average_answer_length(user_id):
    user_answers = disc_ans[disc_ans['umich_discussions_user_id'] == user_id]
    # Handle the case where there are no answers for the user
    if len(user_answers) == 0:
        return 0  
    total_length = user_answers['discussion_answer_content'].str.len().sum()
    return total_length / len(user_answers)

df['average_answer_length'] = df['umich_user_id'].apply(calculate_average_answer_length)

Get the total number of votes given to discussion answers by a user

In [123]:
user_vote_counts = disc_ans_votes.groupby('umich_discussions_user_id')['discussion_answer_vote_value'].sum().reset_index(name='total_votes_given_answers')
df = pd.merge(df, user_vote_counts, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')
df.drop('umich_discussions_user_id', axis=1, inplace=True)
df['total_votes_given_answers'].fillna(0, inplace=True)

Get the total number of votes received on discussion answers by a user

In [124]:
answer_vote_counts_received = disc_ans_votes.groupby('discussion_answer_id')['discussion_answer_vote_value'].sum().reset_index(name='total_votes_received_answers')
discussion_answers_with_votes = pd.merge(disc_ans, answer_vote_counts_received, on='discussion_answer_id', how='left')
discussion_answers_with_votes = discussion_answers_with_votes[['umich_discussions_user_id', 'total_votes_received_answers']]
votes = pd.merge(df, discussion_answers_with_votes, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')
votes.drop('umich_discussions_user_id', axis=1, inplace=True)
votes['total_votes_received_answers'].fillna(0, inplace=True)
votes = votes.groupby('umich_user_id')['total_votes_received_answers'].sum()
df = pd.merge(df, votes, on='umich_user_id', how='left')


Get the total number of discussion questions a user has posted

In [125]:
questions_counts = disc_qs.groupby('umich_discussions_user_id').size().reset_index(name='question_count')
df = pd.merge(df, questions_counts, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')
df.drop('umich_discussions_user_id', axis=1, inplace=True)
df['question_count'].fillna(0, inplace=True)

Get the average length of a discussion question by a user

In [126]:
# Function to calculate the average answer length for a user
def calculate_average_question_length(user_id):
    user_questions = disc_qs[disc_qs['umich_discussions_user_id'] == user_id]
    # Handle the case where there are no answers for the user
    if len(user_questions) == 0:
        return 0  
    total_length = user_questions['discussion_question_details'].str.len().sum()
    return total_length / len(user_questions)

df['average_question_length'] = df['umich_user_id'].apply(calculate_average_question_length)

Get the total number of votes given to discussion questions by a user

In [127]:
user_vote_counts = disc_qs_votes.groupby('umich_discussions_user_id')['discussion_question_vote_value'].sum().reset_index(name='total_votes_given_questions')
df = pd.merge(df, user_vote_counts, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')
df.drop('umich_discussions_user_id', axis=1, inplace=True)
df['total_votes_given_questions'].fillna(0, inplace=True)

Get the total number of votes received on discussion questions by user

In [128]:
question_vote_counts_received = disc_qs_votes.groupby('discussion_question_id')['discussion_question_vote_value'].sum().reset_index(name='total_votes_received_questions')
discussion_questions_with_votes = pd.merge(disc_ans, question_vote_counts_received, on='discussion_question_id', how='left')
discussion_questions_with_votes = discussion_questions_with_votes[['umich_discussions_user_id', 'total_votes_received_questions']]
votes = pd.merge(df, discussion_questions_with_votes, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')
votes.drop('umich_discussions_user_id', axis=1, inplace=True)
votes['total_votes_received_questions'].fillna(0, inplace=True)
votes = votes.groupby('umich_user_id')['total_votes_received_questions'].sum()
df = pd.merge(df, votes, on='umich_user_id', how='left')


Get the number of discussion questions a user is following

In [129]:
merged_df = pd.merge(df, disc_qs_followings, left_on='umich_user_id', right_on='umich_discussions_user_id', how='left')

# Group by 'umich_discussions_user_id' and count the number of questions followed by each user
user_question_counts = merged_df.groupby('umich_discussions_user_id')['discussion_question_following_active'].count().reset_index()

# Rename the columns
user_question_counts.columns = ['umich_user_id', 'total_questions_following']

# Merge user_question_counts back into the final_df 
df = pd.merge(df, user_question_counts, on='umich_user_id', how='left')

# Replace NaN values in the 'total_questions_following' column with 0
df['total_questions_following'].fillna(0, inplace=True)

In [130]:
target.drop('id', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['country_cd', 'educational_attainment'])

# onehot_encoded = pd.get_dummies(df['gender_508'], prefix='gender', dummy_na=True)
# df = pd.concat([df, onehot_encoded], axis=1)

In [131]:
intl_countries = ['country_cd_AE', 'country_cd_AG', 'country_cd_AT',
       'country_cd_AU', 'country_cd_BD', 'country_cd_BE', 'country_cd_CA',
       'country_cd_CN', 'country_cd_FR', 'country_cd_GB', 'country_cd_GR',
       'country_cd_GT', 'country_cd_GY', 'country_cd_HK', 'country_cd_ID',
       'country_cd_IL', 'country_cd_IN', 'country_cd_IQ', 'country_cd_IT',
       'country_cd_JO', 'country_cd_JP', 'country_cd_KR', 'country_cd_KW',
       'country_cd_KY', 'country_cd_LC', 'country_cd_MX', 'country_cd_NG',
       'country_cd_NL', 'country_cd_NZ', 'country_cd_OM', 'country_cd_PK',
       'country_cd_PL', 'country_cd_PT', 'country_cd_PY', 'country_cd_SG',
       'country_cd_SL', 'country_cd_SX', 'country_cd_SZ', 'country_cd_TH',
       'country_cd_UA', 'country_cd_VG', 'country_cd_VI',
       'country_cd_ZA']
df.drop(intl_countries, axis=1, inplace=True)

In [132]:
df.rename(columns={'country_cd_US': 'US'}, inplace=True)

In [133]:
df['intl'] = ~df['US']

In [134]:
# df.drop('gender_Prefer not to say', axis=1, inplace=True)
# df.rename(columns={'gender_Female': 'female', 'gender_Male' : 'male', 'gender_Non-binary / Third gender' : 'non-binary',  'gender_nan' : 'gender_na'}, inplace=True)

In [135]:
df['bachelor_obtained'] = df[['educational_attainment_BACHELOR_DEGREE',
       'educational_attainment_DOCTORATE_DEGREE',
       'educational_attainment_MASTERS_DEGREE',
       'educational_attainment_PROFESSIONAL_DEGREE']].any(axis=1).astype(int)

In [136]:
df['no_bachelor_obtained'] = df[['educational_attainment_ASSOCIATE_DEGREE',
       'educational_attainment_COLLEGE_NO_DEGREE',
       'educational_attainment_HIGH_SCHOOL_DIPLOMA',
       'educational_attainment_LESS_THAN_HIGH_SCHOOL_DIPLOMA']].any(axis=1).astype(int)

In [137]:
df["education_na"] = np.where((df['bachelor_obtained'] == 0) & (df['no_bachelor_obtained'] == 0), 1, 0)

In [138]:
df.drop(['educational_attainment_ASSOCIATE_DEGREE',
       'educational_attainment_BACHELOR_DEGREE',
       'educational_attainment_COLLEGE_NO_DEGREE',
       'educational_attainment_DOCTORATE_DEGREE',
       'educational_attainment_HIGH_SCHOOL_DIPLOMA',
       'educational_attainment_LESS_THAN_HIGH_SCHOOL_DIPLOMA',
       'educational_attainment_MASTERS_DEGREE',
       'educational_attainment_PROFESSIONAL_DEGREE'], axis=1, inplace=True)

In [139]:
columns_to_check = ['Prefer not to say', 'Prefer to self-describe']

# Use the logical OR operator to create the condition
condition = (ethnicity[columns_to_check] == 1.0).any(axis=1)

# Use boolean indexing to drop the rows that meet the condition
ethnicity = ethnicity[~condition]

In [140]:
# col = "White"

# count_ones = ethnicity[col].eq(1.0).sum()
# count_zeros = ethnicity[col].eq(0.0).sum()
# count_nan = ethnicity[col].isna().sum()
# print(count_ones, count_zeros, count_nan)

In [141]:
# survey['gender_508'].eq('Non-binary / Third gender').sum()

In [142]:
df["male"] = (df["reported_or_inferred_gender"] == "male") | (df["gender_508"] == "Male")
df["female"] = (df["reported_or_inferred_gender"] == "female") | (df["gender_508"] == "Female")
df["gender_other"] = (df["gender_508"] == "Non-binary / Third gender")
df["gender_na"] = (df["reported_or_inferred_gender"].isna()) & (df["gender_508"].isna())

In [143]:
df.drop(["reported_or_inferred_gender", "gender_508"], axis=1, inplace=True)

In [144]:
ethnicity_drop_columns = ['Unnamed: 0',
       'essentials_of_social_welfare_policy_user_id', 'completed',
       'phoenix_session_user_id', 'outcome_label_completed',
       'Prefer not to say', 'Prefer to self-describe']
ethnicity.drop(ethnicity_drop_columns, axis=1, inplace=True)

ethnicity["race_others"] = ethnicity[['North American Indigenous', 'Hawaiian & Pacific Islander', 'Middle Eastern or North African']].any(axis=1).astype(int)
race_columns = ['White', 'Black', 'Asian', 'Hispanic, Latino, or Spanish origin', 'North American Indigenous', 'Hawaiian & Pacific Islander', 'Middle Eastern or North African']
ethnicity['race_na'] = ethnicity[race_columns].isna().all(axis=1).astype(int)

ethnicity.drop(['North American Indigenous', 'Hawaiian & Pacific Islander', 'Middle Eastern or North African'], axis=1, inplace=True)

In [145]:
df = pd.merge(df, ethnicity, on='umich_user_id', how='left')
df = df.rename(columns={'Hispanic, Latino, or Spanish origin': 'latinx', 'Black': 'black', 'Asian': 'asian', 'White': 'white'})
df.fillna(0, inplace=True)

In [146]:
df[['US', 'intl', 'male', 'female', 'gender_other', 'gender_na', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'latinx', 'black', 'asian', 'white', 'race_others', 'race_na']] = df[['US', 'intl', 'male', 'female', 'gender_other', 'gender_na', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'latinx', 'black', 'asian', 'white', 'race_others', 'race_na']].astype(int)

In [147]:
target = target[['umich_user_id', 'completed']]
df = pd.merge(df, target, on="umich_user_id", how="left")

In [148]:
df.to_csv('./data/preprocessed.csv')

In [149]:
df['female'].sum()

759

In [150]:
test = pd.read_csv('./data/preprocessed.csv')
# test['bachelor_obtained'].sum() + test['no_bachelor_obtained'].sum() + test['education_na'].sum()
test.drop('umich_user_id', axis=1, inplace=True)
test.mean()

Unnamed: 0                        565.500000
answer_count                        4.326855
average_answer_length             287.235493
total_votes_given_answers           0.730565
total_votes_received_answers        0.786219
question_count                      0.120141
average_question_length            13.694562
total_votes_given_questions         0.009717
total_votes_received_questions      0.417845
total_questions_following           0.060954
US                                  0.889576
intl                                0.110424
bachelor_obtained                   0.458481
no_bachelor_obtained                0.053887
education_na                        0.487633
male                                0.160777
female                              0.670495
gender_other                        0.004417
gender_na                           0.166961
white                               0.337456
latinx                              0.042403
black                               0.082155
asian     

### Logistic Regression

In [151]:
# X = final_df.drop('completed', axis=1)
# y = final_df['completed']

# model = LogisticRegression()

# scores = cross_val_score(model, X, y, cv=10)

# for i, score in enumerate(scores, 1):
#     print(f'Fold {i}: Accuracy = {score:.2f}')


# mean_accuracy = scores.mean()
# std_accuracy = scores.std()
# print(f'Mean Accuracy = {mean_accuracy:.2f}')
# print(f'Standard Deviation = {std_accuracy:.2f}')

In [152]:
# model.fit(X, y)
# coefficients = model.coef_
# coefficients_dict = (np.std(X, 0)*(model.coef_[0])).to_dict()
# # print(model.coef_)
# # coefficients_dict = dict(zip(X.columns, coefficients[0]))
# print(coefficients_dict)

In [153]:
# len(coefficients_dict)

In [154]:
# sorted(list(coefficients_dict.items()), key=lambda x : abs(x[1]), reverse=True)

In [155]:
# X = X-np.mean(X, axis=0)
# X = X/np.std(X, axis=0)
# X

In [156]:
# scores = cross_val_score(model, X, y, cv=10)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# # for i, score in enumerate(scores, 1):
# #     print(f'Fold {i}: Accuracy = {score:.2f}')

# # mean_accuracy = scores.mean()
# # std_accuracy = scores.std()

# # print(f'Mean Accuracy = {mean_accuracy:.2f}')
# # print(f'Standard Deviation = {std_accuracy:.2f}')

# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# auc = roc_auc_score(y_test, y_pred)

# # Print the AUC score
# print(f'AUC: {auc:.2f}')

# coefficients = model.coef_
# coefficients_dict = (np.std(X, 0)*(model.coef_[0])).to_dict()

# sorted(list(coefficients_dict.items()), key=lambda x : abs(x[1]), reverse=True)

In [157]:
# final_df = pd.read_csv('test.csv')

In [158]:
# df.columns

In [159]:
# df['educational_attainment_BACHELOR_DEGREE'].value_counts()

In [160]:
# # Calculate the confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred)

# # Extract FP and TN from the confusion matrix
# FP = conf_matrix[0, 1]  # False Positives
# TN = conf_matrix[1, 1]  # True Negatives

# # Calculate FPR
# FPR = FP / (FP + TN)
# print("False Positive Rate:", FPR)

In [161]:
# X = df.drop('completed', axis=1)
# y = df['completed']

# model = LogisticRegression()

# kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# aucs = []
# fprs = []

# # print(X.shape, y.shape)
# # print(y)

# y = np.array(y)
# X = np.array(X)
# for train_index, test_index in kfold.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     auc = roc_auc_score(y_test, y_pred)
#     aucs.append(auc)

#     # Print the AUC score
#     # print(f'AUC: {auc:.2f}')
    
#     # Train and evaluate your model on the current fold
#     # You can replace this with your model training and evaluation code
#     conf_matrix = confusion_matrix(y_test, y_pred)

#     # Extract FP and TN from the confusion matrix
#     FP = conf_matrix[0, 1]  # False Positives
#     TN = conf_matrix[1, 1]  # True Negatives

#     # Calculate FPR
#     FPR = FP / (FP + TN)
#     fprs.append(FPR)
#     # print("False Positive Rate:", FPR)

# auc_avg = np.average(aucs)
# auc_std = np.std(aucs)
# fpr_avg = np.average(fprs)
# fpr_std = np.std(fprs)


In [162]:
# print(f'AUC: {auc_avg:.2f} +/- {auc_std:.2f}')
# print(f'FPR: {fpr_avg:.2f} +/- {fpr_std:.2f}')